In [86]:
!pip install langchain openai pypdf
!pip install -U langchain-community
!pip install tiktoken
!pip install -U langchain-huggingface huggingface-hub
!pip install sentence-transformers
!pip install tf-keras
!pip install hf_xet

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 6.9 MB/s eta 0:00:00


In [1]:
#api key tanımı

import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "*************************" 


In [88]:
#pdften metni yükleme

from langchain_community.document_loaders import PyPDFLoader
pdf_path="Forex_Price_Prediction_After_Economic_News_with_Boomerang_Strategy_and_Machine_Learning.pdf"

loader=PyPDFLoader(pdf_path)
documents=loader.load()

print("PDF başarıyla yüklendi. Toplam sayfa:", len(documents))
print(documents[0].page_content[:300])

PDF başarıyla yüklendi. Toplam sayfa: 7
Forex Price Prediction After Economic News with Boomerang Strategy and Machine Learning 
 
Batuhan ÇELİKBAŞ Computer Engineering Department of Karabuk University,  
batuhancelikbass1@gmail.com 
Oğuz FINDIK,Computer Engineering Department of Karabuk University,  
oguzfindik@karabuk.edu.tr
ABSTRACT— T


In [89]:
#embedding, retriver, vektör kurulumu
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
db = FAISS.from_documents(documents, embeddings)
retriever = db.as_retriever()

In [110]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain_huggingface.llms import HuggingFaceEndpoint
from langchain.chains import RetrievalQA

llm_name = "google/flan-t5-base"
llm_pipe = pipeline("text2text-generation", model=llm_name)

def pdf_qa_offline(question):
    docs = retriever.get_relevant_documents(question)
    input_text = docs[0].page_content[:800]
    prompt = (
         f"Answer the following question in detail based only on the provided context.\n"
         f"Context: {input_text}\n"
         f"Question: {question}\n"
         f"Answer:"
    )
    answer = llm_pipe(prompt, max_new_tokens=200)[0]['generated_text']
    return answer

soru = "Which deep learning models are discussed in the PDF?"
cevap = pdf_qa_offline(soru)
print("answer:", cevap)

Device set to use cpu


answer: H. Deep Learning Architectures Each model used in this study takes OHLC (Open, High, Low, Close) data containing 10-time steps from the past as input and produces four-dimensional (opening, highest, lowest, and closing) price estimates for the next time step based on this historical information. Model architectures generally consist of two consecutive recurrent layers (LSTM, GRU, or BiLSTM) and a Dense(4) output layer following these layers. In the first recurrent layer, the “return_sequences=True” parameter is activated so that the output can be transferred sequentially to the second layer; thus, the models can better learn dynamic structures over time. During training, all models were optimized with the Mean Squared Error (MSE) loss function due to the continuous value es
